In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import warnings
from sklearn import linear_model, tree, discriminant_analysis, naive_bayes, ensemble, gaussian_process
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, FunctionTransformer, LabelEncoder, RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, RepeatedStratifiedKFold
from sklearn.metrics import log_loss, confusion_matrix
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 100)
from pycaret.classification import *
from function.data_engineering import *
from sklearn.svm import SVC


from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

In [3]:
data = pd.read_csv('penyisihan-satria-data-itb-2023/train.csv')
test = pd.read_csv('penyisihan-satria-data-itb-2023/test.csv')
data.index = data.id
test.index = test.id
data = data.iloc[:,1:]
test = test.iloc[:,1:]
or_cols = [c for c in data.columns if c != 'target']
int_cols = [c for c in data.columns if data[c].dtype=='int64' and c != 'target']
float_cols = [c for c in data.columns if data[c].dtype=='float64' and c != 'target']

In [4]:
h_columns = [c for c in data.columns if c[-1]=='h']
d_columns = [c for c in data.columns if c[-1]=='d']

In [5]:
abs(data.corr()['target']).sort_values(ascending=False)[1:]

bb_a_hh        0.409673
hst            0.404691
iwh            0.320410
psch           0.319234
psh            0.312713
bb_av_h        0.309727
vch            0.307735
bwh            0.307491
whh            0.303591
b365h          0.303146
bb_mx_h        0.298787
bb_av_2_5_1    0.257839
iwd            0.256921
bb_mx_2_5_1    0.255355
whd            0.252662
bb_av_d        0.249135
bwd            0.247222
b365d          0.246322
vcd            0.245664
pscd           0.245553
psd            0.244675
bb_mx_d        0.244367
bb_av_2_5      0.208117
bb_mx_2_5      0.204625
hs             0.203025
hf             0.154112
hy             0.125226
bb_ou          0.123818
hr             0.121942
bb_ah          0.073200
bb_mx_ahh      0.055756
bb_av_ahh      0.045254
bb_av_aha      0.034883
bb_mx_aha      0.026466
hc             0.013607
Name: target, dtype: float64

In [6]:
data[h_columns]

,b365h,bwh,iwh,psh,whh,vch,bb_mx_h,bb_av_h,bb_ah,bb_a_hh,bb_mx_ahh,bb_av_ahh,psch
id,,,,,,,,,,,,,
1,1.95,1.91,1.90,1.93,1.85,1.95,2.02,1.94,22,-0.75,2.26,2.21,1.96
2,1.28,1.26,1.27,1.30,1.29,1.29,1.32,1.28,20,-1.75,2.12,2.06,1.37
3,1.28,1.28,1.33,1.27,1.25,1.29,1.33,1.27,23,-1.50,1.87,1.78,1.26
4,1.90,1.83,1.90,1.91,1.91,1.90,1.98,1.89,19,-0.50,1.94,1.88,2.08
5,2.50,2.50,2.50,2.52,2.50,2.50,2.57,2.47,20,-0.25,2.15,2.10,2.73
...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,1.55,1.57,1.50,1.58,1.55,1.57,1.60,1.57,21,-1.00,2.05,1.98,1.53
597,1.16,1.16,1.22,1.18,1.17,1.18,1.22,1.17,21,-2.00,1.91,1.84,1.15
598,1.90,1.90,1.95,1.91,1.91,1.85,1.95,1.89,19,-0.50,1.91,1.86,1.89


In [7]:
tmp = data.copy()
tmp['ratio_hs'] = data['hst']/data['hs']
tmp[['ratio_hs','target']].corr()

,ratio_hs,target
ratio_hs,1.00000,0.31859
target,0.31859,1.00000


In [8]:
tmp['h_max'] = tmp[h_columns[:6]].max(axis=1)
tmp['h_mean'] = tmp[h_columns[:6]].mean(axis=1)
tmp['h_min'] = tmp[h_columns[:6]].min(axis=1)
tmp['d_max'] = tmp[d_columns[:6]].max(axis=1)
tmp['d_mean'] = tmp[d_columns[:6]].mean(axis=1)
tmp['d_min'] = tmp[d_columns[:6]].min(axis=1)
abs(tmp.corr()['target']).sort_values(ascending=False)[1:]

bb_a_hh        0.409673
hst            0.404691
h_min          0.320534
iwh            0.320410
psch           0.319234
ratio_hs       0.318590
psh            0.312713
h_mean         0.309952
bb_av_h        0.309727
vch            0.307735
bwh            0.307491
whh            0.303591
b365h          0.303146
h_max          0.301330
bb_mx_h        0.298787
bb_av_2_5_1    0.257839
iwd            0.256921
bb_mx_2_5_1    0.255355
d_min          0.254191
whd            0.252662
d_mean         0.249979
bb_av_d        0.249135
bwd            0.247222
b365d          0.246322
vcd            0.245664
pscd           0.245553
psd            0.244675
bb_mx_d        0.244367
d_max          0.243256
bb_av_2_5      0.208117
bb_mx_2_5      0.204625
hs             0.203025
hf             0.154112
hy             0.125226
bb_ou          0.123818
hr             0.121942
bb_ah          0.073200
bb_mx_ahh      0.055756
bb_av_ahh      0.045254
bb_av_aha      0.034883
bb_mx_aha      0.026466
hc             0

In [9]:
X = [c for c in tmp.columns if c != 'target']
scale = MinMaxScaler()
scale.fit(tmp[X])
tmp[X] = scale.transform(tmp[X])

In [10]:
classifiers = [

    #GLM
    linear_model.LogisticRegressionCV(),

    #Navies Bayes
    naive_bayes.BernoulliNB(),
    naive_bayes.GaussianNB(),

    #Discriminant Analysis
    discriminant_analysis.LinearDiscriminantAnalysis(),
    discriminant_analysis.QuadraticDiscriminantAnalysis(),

    #Ensemble Methods
    ensemble.AdaBoostClassifier(),
    ensemble.BaggingClassifier(),
    ensemble.ExtraTreesClassifier(),
    ensemble.GradientBoostingClassifier(),
    ensemble.RandomForestClassifier(),

    #Gaussian Processes
    gaussian_process.GaussianProcessClassifier(),

    #xgboost: http://xgboost.readthedocs.io/en/latest/model.html
#     xgb.XGBClassifier()    
]

def find_best_algorithms(classifier_list, X, y):
    # This function is adapted from https://www.kaggle.com/yassineghouzam/titanic-top-4-with-ensemble-modeling
    # Cross validate model with Kfold stratified cross validation
    kfold = StratifiedKFold(n_splits=5)

    # Grab the cross validation scores for each algorithm
    cv_results = [cross_val_score(classifier, X, y, scoring = "accuracy", cv = kfold) for classifier in classifier_list]
    cv_means = [cv_result.mean() * -1 for cv_result in cv_results]
    cv_std = [cv_result.std() for cv_result in cv_results]
    algorithm_names = [alg.__class__.__name__ for alg in classifiers]

    # Create a DataFrame of all the CV results
    cv_results = pd.DataFrame({
        "Accuracy": cv_means,
        "Accuracy Std": cv_std,
        "Algorithm": algorithm_names
    }).sort_values(by='Accuracy')
    return cv_results

In [11]:
var = ['bb_a_hh','hst','h_min','iwh','psch','ratio_hs','psh','h_mean','bb_av_h','vch','bwh','whh','b365h','h_max','bb_mx_h','bb_av_2_5_1','iwd','bb_mx_2_5_1','d_min','whd','d_mean','bb_av_d','bwd','b365d']
algorithm_results = find_best_algorithms(classifiers, tmp.loc[:,var], tmp.loc[:,'target'])
algorithm_results

,Accuracy,Accuracy Std,Algorithm
3,-0.716667,0.038370,LinearDiscriminantAnalysis
0,-0.703333,0.048477,LogisticRegressionCV
10,-0.691667,0.045031,GaussianProcessClassifier
8,-0.675000,0.044721,GradientBoostingClassifier
6,-0.671667,0.034801,BaggingClassifier
9,-0.671667,0.048762,RandomForestClassifier
5,-0.670000,0.038944,AdaBoostClassifier
7,-0.658333,0.051908,ExtraTreesClassifier
2,-0.640000,0.037417,GaussianNB
4,-0.615000,0.021985,QuadraticDiscriminantAnalysis


In [12]:
var_ = var.copy()
var_.append('target')
exp_name = setup(data = tmp[var_], target = 'target',fold=5,session_id = 42, )
exp_name

,Description,Value
0,Session id,42
1,Target,target
2,Target type,Binary
3,Original data shape,"(600, 25)"
4,Transformed data shape,"(600, 25)"
5,Transformed train set shape,"(420, 25)"
6,Transformed test set shape,"(180, 25)"
7,Numeric features,24
8,Preprocess,True
9,Imputation type,simple


In [13]:
var_

['bb_a_hh',
 'hst',
 'h_min',
 'iwh',
 'psch',
 'ratio_hs',
 'psh',
 'h_mean',
 'bb_av_h',
 'vch',
 'bwh',
 'whh',
 'b365h',
 'h_max',
 'bb_mx_h',
 'bb_av_2_5_1',
 'iwd',
 'bb_mx_2_5_1',
 'd_min',
 'whd',
 'd_mean',
 'bb_av_d',
 'bwd',
 'b365d',
 'target']

In [14]:
cross_validation(tmp.loc[:,X],tmp['target'],discriminant_analysis.LinearDiscriminantAnalysis())

              precision    recall  f1-score   support

           0       0.69      0.68      0.69        60
           1       0.69      0.70      0.69        60

    accuracy                           0.69       120
   macro avg       0.69      0.69      0.69       120
weighted avg       0.69      0.69      0.69       120
 0.6916452531425793
              precision    recall  f1-score   support

           0       0.68      0.87      0.76        60
           1       0.82      0.60      0.69        60

    accuracy                           0.73       120
   macro avg       0.75      0.73      0.73       120
weighted avg       0.75      0.73      0.73       120
 0.7285067873303168
              precision    recall  f1-score   support

           0       0.64      0.75      0.69        60
           1       0.70      0.58      0.64        60

    accuracy                           0.67       120
   macro avg       0.67      0.67      0.66       120
weighted avg       0.67      0.67   

[0.6916452531425793,
 0.7285067873303168,
 0.6643356643356644,
 0.7165879410947485,
 0.7499305362600722]

In [15]:

cross_validation(data.iloc[:,:-1],data['target'],naive_bayes.GaussianNB())

              precision    recall  f1-score   support

           0       0.58      0.82      0.68        60
           1       0.69      0.40      0.51        60

    accuracy                           0.61       120
   macro avg       0.63      0.61      0.59       120
weighted avg       0.63      0.61      0.59       120
 0.590562613430127
              precision    recall  f1-score   support

           0       0.68      0.63      0.66        60
           1       0.66      0.70      0.68        60

    accuracy                           0.67       120
   macro avg       0.67      0.67      0.67       120
weighted avg       0.67      0.67      0.67       120
 0.6662958843159066
              precision    recall  f1-score   support

           0       0.62      0.80      0.70        60
           1       0.71      0.50      0.59        60

    accuracy                           0.65       120
   macro avg       0.66      0.65      0.64       120
weighted avg       0.66      0.65    

[0.590562613430127,
 0.6662958843159066,
 0.6419437340153452,
 0.6577391304347826,
 0.7037037037037037]

In [16]:
cross_validation(data.iloc[:,:-1],data['target'],SVC(gamma='scale',kernel='rbf',random_state=0))

              precision    recall  f1-score   support

           0       0.62      0.77      0.69        60
           1       0.70      0.53      0.60        60

    accuracy                           0.65       120
   macro avg       0.66      0.65      0.65       120
weighted avg       0.66      0.65      0.65       120
 0.6451703745423825
              precision    recall  f1-score   support

           0       0.68      0.88      0.77        60
           1       0.83      0.58      0.69        60

    accuracy                           0.73       120
   macro avg       0.76      0.73      0.73       120
weighted avg       0.76      0.73      0.73       120
 0.7271952259164536
              precision    recall  f1-score   support

           0       0.62      0.82      0.71        60
           1       0.73      0.50      0.59        60

    accuracy                           0.66       120
   macro avg       0.68      0.66      0.65       120
weighted avg       0.68      0.66   

[0.6451703745423825,
 0.7271952259164536,
 0.6495476885818079,
 0.6799077211448344,
 0.7394775544505917]

In [17]:
tmp[var_]

,bb_a_hh,hst,h_min,iwh,psch,ratio_hs,psh,h_mean,bb_av_h,vch,bwh,whh,b365h,h_max,bb_mx_h,bb_av_2_5_1,iwd,bb_mx_2_5_1,d_min,whd,d_mean,bb_av_d,bwd,b365d,target
id,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.409091,0.400000,0.047198,0.049025,0.037553,0.260870,0.042074,0.044468,0.044489,0.045113,0.043108,0.044568,0.040565,0.040128,0.034918,0.137184,0.060773,0.121212,0.065934,0.059406,0.046289,0.048077,0.054054,0.035714,0
2,0.227273,0.333333,0.012389,0.011813,0.012658,0.238095,0.011252,0.011464,0.011122,0.012030,0.010526,0.013370,0.010027,0.010488,0.008915,0.407942,0.314917,0.357576,0.318681,0.306931,0.275496,0.259615,0.279279,0.250000,0
3,0.272727,0.400000,0.011799,0.015357,0.008017,0.352941,0.009785,0.011551,0.010617,0.012030,0.011529,0.011142,0.010027,0.011856,0.009287,0.404332,0.281768,0.378788,0.285714,0.306931,0.266388,0.263112,0.256757,0.250000,0
4,0.454545,0.733333,0.046018,0.049025,0.042616,0.423077,0.041096,0.043252,0.041962,0.042607,0.039098,0.047911,0.038286,0.038304,0.033432,0.238267,0.060773,0.212121,0.065934,0.084158,0.063760,0.064685,0.076577,0.056429,0
5,0.500000,0.400000,0.085546,0.084465,0.070042,0.300000,0.070939,0.075126,0.071284,0.072682,0.072682,0.080780,0.065634,0.066119,0.055349,0.061372,0.016575,0.051515,0.021978,0.019802,0.013140,0.014860,0.018018,0.014286,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,0.363636,0.333333,0.026549,0.025399,0.019409,0.312500,0.024951,0.025621,0.025784,0.026065,0.026065,0.027855,0.022334,0.023256,0.019316,0.162455,0.149171,0.142424,0.120879,0.108911,0.098552,0.095280,0.099099,0.085714,1
597,0.181818,0.800000,0.006490,0.008860,0.003376,0.571429,0.005382,0.006080,0.005561,0.006516,0.005514,0.006685,0.004558,0.006840,0.005201,0.501805,0.392265,0.424242,0.395604,0.405941,0.403464,0.395105,0.414414,0.392857,1
598,0.454545,0.533333,0.047198,0.051979,0.034599,0.533333,0.041096,0.043860,0.041962,0.040100,0.042607,0.047911,0.038286,0.040128,0.032318,0.083032,0.055249,0.075758,0.032967,0.029703,0.039122,0.042832,0.045045,0.035714,1


In [18]:
for col in var:
  tmp_train = freq_encoding(tmp, col)
tmp_train

,hs,hst,hf,hc,hy,hr,b365h,b365d,bwh,bwd,iwh,iwd,psh,psd,whh,whd,vch,vcd,bb_mx_h,bb_av_h,bb_mx_d,bb_av_d,bb_ou,bb_mx_2_5,bb_av_2_5,bb_mx_2_5_1,bb_av_2_5_1,bb_ah,bb_a_hh,bb_mx_ahh,bb_av_ahh,bb_mx_aha,bb_av_aha,psch,pscd,target,ratio_hs,h_max,h_mean,h_min,d_max,d_mean,d_min,bb_a_hh_encode,hst_encode,h_min_encode,iwh_encode,psch_encode,ratio_hs_encode,psh_encode,h_mean_encode,bb_av_h_encode,vch_encode,bwh_encode,whh_encode,b365h_encode,h_max_encode,bb_mx_h_encode,bb_av_2_5_1_encode,iwd_encode,bb_mx_2_5_1_encode,d_min_encode,whd_encode,d_mean_encode,bb_av_d_encode,bwd_encode,b365d_encode
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.617647,0.400000,0.238095,0.555556,0.333333,0.0,0.040565,0.035714,0.043108,0.054054,0.049025,0.060773,0.042074,0.053289,0.044568,0.059406,0.045113,0.041667,0.034918,0.044489,0.038055,0.048077,0.705882,0.468421,0.491124,0.121212,0.137184,0.8,0.409091,0.522059,0.533835,0.196850,0.216667,0.037553,0.041563,0,0.260870,0.040128,0.044468,0.047198,0.038020,0.046289,0.065934,0.068333,0.108333,0.008333,0.035000,0.003333,0.006667,0.005000,0.001667,0.006667,0.016667,0.010000,0.016667,0.015000,0.006667,0.006667,0.011667,0.070000,0.020000,0.076667,0.078333,0.001667,0.010000,0.065000,0.085000
2,0.558824,0.333333,0.142857,0.444444,0.166667,0.0,0.010027,0.250000,0.010526,0.279279,0.011813,0.314917,0.011252,0.265612,0.013370,0.306931,0.012030,0.250000,0.008915,0.011122,0.242424,0.259615,0.588235,0.168421,0.177515,0.357576,0.407942,0.6,0.227273,0.419118,0.421053,0.283465,0.300000,0.012658,0.196176,0,0.238095,0.010488,0.011464,0.012389,0.246772,0.275496,0.318681,0.035000,0.136667,0.005000,0.013333,0.006667,0.003333,0.008333,0.001667,0.015000,0.023333,0.016667,0.018333,0.020000,0.013333,0.005000,0.008333,0.003333,0.003333,0.005000,0.041667,0.001667,0.001667,0.021667,0.033333
3,0.441176,0.400000,0.285714,0.388889,0.166667,0.0,0.010027,0.250000,0.011529,0.256757,0.015357,0.281768,0.009785,0.260616,0.011142,0.306931,0.012030,0.250000,0.009287,0.010617,0.242424,0.263112,0.529412,0.163158,0.171598,0.378788,0.404332,0.9,0.272727,0.235294,0.210526,0.519685,0.533333,0.008017,0.267664,0,0.352941,0.011856,0.011551,0.011799,0.246772,0.266388,0.285714,0.058333,0.108333,0.025000,0.015000,0.011667,0.005000,0.015000,0.001667,0.005000,0.023333,0.011667,0.026667,0.020000,0.011667,0.011667,0.001667,0.025000,0.003333,0.028333,0.041667,0.001667,0.001667,0.025000,0.033333
4,0.705882,0.733333,0.476190,0.333333,0.166667,0.0,0.038286,0.056429,0.039098,0.076577,0.049025,0.060773,0.041096,0.064113,0.047911,0.084158,0.042607,0.062500,0.033432,0.041962,0.059197,0.064685,0.764706,0.321053,0.319527,0.212121,0.238267,0.5,0.454545,0.286765,0.285714,0.417323,0.441667,0.042616,0.055694,0,0.423077,0.038304,0.043252,0.046018,0.052367,0.063760,0.065934,0.026667,0.025000,0.013333,0.035000,0.005000,0.001667,0.003333,0.003333,0.005000,0.003333,0.010000,0.018333,0.021667,0.005000,0.003333,0.006667,0.070000,0.018333,0.076667,0.040000,0.001667,0.011667,0.026667,0.016667
5,0.529412,0.400000,0.428571,0.166667,0.500000,0.0,0.065634,0.014286,0.072682,0.018018,0.084465,0.016575,0.070939,0.018318,0.080780,0.019802,0.072682,0.016667,0.055349,0.071284,0.009866,0.014860,0.470588,0.668421,0.692308,0.051515,0.061372,0.6,0.500000,0.441176,0.451128,0.259843,0.291667,0.070042,0.017456,0,0.300000,0.066119,0.075126,0.085546,0.010043,0.013140,0.021978,0.301667,0.108333,0.031667,0.023333,0.005000,0.031667,0.003333,0.001667,0.011667,0.023333,0.018333,0.040000,0.026667,0.001667,0.003333,0.008333,0.038333,0.018333,0.101667,0.103333,0.001667,0.010000,0.086667,0.058333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,0.411765,0.333333,0.238095,0.277778,0.000000,0.0,0.022334,0.085714,0.026065,0.099099,0.025399,0.149171,0.024951,0.094921,0.0

In [19]:
steps = [('pca', PCA(n_components=15)), ('m', discriminant_analysis.LinearDiscriminantAnalysis())]
model = Pipeline(steps=steps)


cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=0)
n_scores = cross_val_score(model, tmp_train[var], tmp['target'], scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

Accuracy: 0.701 (0.047)


In [20]:
import xgboost as xgb
from scipy.stats import randint, uniform
from sklearn.model_selection import RandomizedSearchCV

# Define the parameter grid

# Define the parameter distributions for randomized search
param_dist = {
    'max_depth': randint(3, 8),
    'learning_rate': uniform(0.001, 0.1),
    'n_estimators': randint(100, 1000),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4),
}


# Create an XGBoost classifier
xgb_model = xgb.XGBClassifier()

random_search = RandomizedSearchCV(
    xgb_model,
    param_distributions=param_dist,
    n_iter=100,
    cv=5,
    scoring='accuracy',
    random_state=42
)

# Fit the RandomizedSearchCV object to the data
random_search.fit(tmp[var], tmp['target'])

# Print the best parameters found
print("Best parameters found:")
print(random_search.best_params_)

# Print the best F1 score found
print("Best F1 score:", random_search.best_score_)

KeyboardInterrupt: 

In [ ]:
tmp[var_].to_csv('updated_data.csv')